In [1]:
# pseduo code
import sys
sys.path.append("D:/ASOML/SNOCONE")
from CNN_memoryOptimization import clear_memory, memory_efficient_prediction, DataGenerator
from CNN_preProcessing import*
from CNN_benchmarks import swe_fsca_consistency_loss_fn, make_swe_fsca_loss, masked_loss_fn, masked_mse, masked_mae, masked_rmse
from CNN_modelArchitectureBlocks import conv_block, identity_block, basic_block, resnet_model_implementation, model_predict, load_and_prepare_model, Baseline_CNN
import rasterio
import shap
import pandas as pd
from rasterio.mask import mask
from rasterio.windows import from_bounds
import psutil
from rasterio.transform import from_bounds 
import numpy as np
import sys
import os
import subprocess
from sklearn.model_selection import train_test_split
from tensorflow.keras.losses import MeanSquaredError, MeanAbsoluteError
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.utils import register_keras_serializable
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils import to_categorical
from keras.models import Sequential, Model
from tensorflow.keras.models import load_model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, Dense, BatchNormalization, Activation, Input, Add
from keras.callbacks import EarlyStopping, ModelCheckpoint
import time
import matplotlib.pyplot as plt
from rasterio.transform import from_bounds
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import Sequence
from tensorflow.keras.utils import register_keras_serializable
from tensorflow.keras.losses import MeanSquaredError, MeanAbsoluteError
from tensorflow.keras.losses import Loss
import gc
import tensorflow.keras.backend as K

start_time = time.time()
print("modules established")

## establish file paths
# years = list(range(2022, 2023))
start_year = 2022
end_year = 2024
Domain = "Rockies"
GPU = "N"

# workspaces
WorkspaceBase = f"D:/ASOML/{Domain}/"
phv_features = WorkspaceBase + "features/scaled/"
tree_workspace = WorkspaceBase + "treeCover/"
land_workspace = WorkspaceBase + "landCover/"
modelOuptuts = WorkspaceBase + "modelOutputs/BasinSpecifics/"
DMFSCAWorkspace = WorkspaceBase + "Rockies_DMFSCA/"
final_activation = 'relu'
architectures = ['Baseline']  # Options are: Baseline, ResNet18, ResNet34, ResNet50, CustomSWE
basin_list = ['BigThompson', 'BlueRiver', 'BoulderCreek', 'ClearCreek', 'Conejos', 'Dolores', 
              'EastRiver', 'GunnisonNorth', 'Poudre', 'RioGrande','RoaringFork', 'SouthPlatte', 
              'StVrainLefthand', 'Taylor', 'UintaMountains', 'WindyGap', 'YampaRiver']

test_groups = [
            ('Group1', 2025, 'G1'),
            ('Group2', 2025, 'G2'), 
            ('Group3', 2025, 'G3'),
            ('Group4', 2025, 'G4'),
            ('Group5', 2025, 'G5'),
            ('Group6', 2025, 'G6')
        ]

_mae_metric = MeanAbsoluteError()
_mse_metric = MeanSquaredError()
_rmse_metric = MeanSquaredError()

# Check GPU availability
if GPU == "Y":
    print("\n Checking and start running on GPU")
    print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
    print("GPU devices:", tf.config.list_physical_devices('GPU'))
    
    # Configure GPU memory growth (prevents TensorFlow from allocating all GPU memory)
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            print("GPU memory growth enabled")
        except RuntimeError as e:
            print(e)
    
    # Verify TensorFlow is using GPU
    with tf.device('/GPU:0'):
        a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
        b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
        c = tf.matmul(a, b)
        print("GPU computation result:", c)

for basin_name in basin_list:
    for architecture in architectures:
        print(architecture)
        # shapeChecks = "N"
        ## seting folder
        from datetime import datetime
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        # create folder for model outputs
        os.makedirs(modelOuptuts + f"{basin_name}_{str(timestamp)}/", exist_ok=True)
        inter_model_outWorkspace = modelOuptuts + f"{str(timestamp)}/"
        
        # f = open(inter_model_outWorkspace + f"code_output_{timestamp}.txt", "a")
        # sys.stdout = f
        print("MODEL OUTPUTS TO BE PRINTED TO THIS DOC")
        X, y, featureNames = target_feature_stacks_basins(start_year=start_year, 
                                                   end_year=end_year, 
                                                   WorkspaceBase=WorkspaceBase, 
                                                   ext = "nonull_fnl.tif", 
                                                   vegetation_path = tree_workspace, 
                                                   landCover_path = land_workspace, 
                                                   phv_path = phv_features , 
                                                   target_shape=(256,256),
                                                   basin_name=basin_name,
                                                   shapeChecks="Y")
        
        print("")
        print("Shape of input data")
        print(f"feature shape: {X.shape}")
        print(f"target shape: {y.shape}")
        feat_shape = X.shape
        featNo = feat_shape[-1]
        print(featNo)
        print(featureNames)
        
        # split between training and test data
        X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.15, shuffle=True)
        print("***")
        print("________________________________ Training and Validation Data Shapes ________________________________")
        print("Training data shape:", X_train.shape, y_train.shape)
        print("Validation data shape:", X_valid.shape, y_valid.shape)
        print("***")
        x_trainShape = X_train.shape
        x_validShape = X_valid.shape
        
        # Assuming featNo and final_activation are defined in your original code
        featNo = featNo  # Replace with your actual feature count
        final_activation = final_activation  # Replace with your actual activation
        
        # Create the model
        model = resnet_model_implementation(featNo, architecture, final_activation)
        
        # Your existing custom loss function
        from tensorflow.keras.losses import MeanSquaredError
        from tensorflow.keras.metrics import MeanAbsoluteError
        
        custom_loss_fn = make_swe_fsca_loss(
            base_loss_fn=MeanSquaredError(),
            penalty_weight=0.3,
            swe_threshold=0.01,
            fsca_threshold=0.01,
            mask_value=-1
        )
        
        # Compile with your existing setup
        model.compile(
            optimizer='adam',
            loss=custom_loss_fn,
            metrics=[masked_rmse, masked_mae, masked_mse]
        )
        
        print(model.summary())
        
        # establish the model
        checkpoint = ModelCheckpoint(
            f"{inter_model_outWorkspace}/best_model_{basin_name}_{timestamp}.weights.h5", 
            monitor="val_loss",
            verbose=1, 
            save_best_only=True, mode='min'
        )
        early_stopping = EarlyStopping(monitor="val_masked_rmse", mode='min', verbose=1, patience=10, restore_best_weights=True)
        
        
        batch_size = 32
        train_generator = DataGenerator(X_train, y_train, batch_size=batch_size)
        valid_generator = DataGenerator(X_valid, y_valid, batch_size=batch_size)
        
        history = model.fit(
            train_generator,
            validation_data=valid_generator,
            epochs=100,
            callbacks=[checkpoint, early_stopping]
        )
    
        ## plotting results 
        fig, axs = plt.subplots(3, 1, figsize=(12, 10), sharex=True)
        # 1. Masked MSE (used as loss)
        axs[0].plot(history.history['loss'], label='Train MSE Loss')
        axs[0].plot(history.history['val_loss'], label='Val MSE Loss')
        axs[0].set_ylabel('MSE Loss')
        axs[0].set_title('Masked MSE')
        axs[0].legend()
        axs[0].grid(True)
        # 2. Masked RMSE
        axs[1].plot(history.history['masked_rmse'], label='Train RMSE')
        axs[1].plot(history.history['val_masked_rmse'], label='Val RMSE')
        axs[1].set_ylabel('RMSE')
        axs[1].set_title('Masked RMSE')
        axs[1].legend()
        axs[1].grid(True)
        # 3. Masked MAE
        axs[2].plot(history.history['masked_mae'], label='Train MAE')
        axs[2].plot(history.history['val_masked_mae'], label='Val MAE')
        axs[2].set_ylabel('MAE')
        axs[2].set_title('Masked MAE')
        axs[2].set_xlabel('Epoch')
        axs[2].legend()
        axs[2].grid(True)
        
        plt.tight_layout()
        plt.savefig(inter_model_outWorkspace + f"{basin_name}_Model_error_epochs.png")
        plt.show()
        
        ## Add metrics to recurring error tracking sheet
        metrics_to_track = ['val_masked_rmse', 'val_masked_mse', 'val_masked_mae']
        best_metrics = {}
        print("\nValidation Metric Progression:")
        for metric in metrics_to_track:
            values = history.history.get(metric, [])
            if values:
                best_val = min(values)
                best_metrics[metric] = best_val
                print(f"{metric}: Start = {values[0]:.4f}, End = {values[-1]:.4f}")
                print(f"{metric}: Best = {best_val:.4f}")
            else:
                print(f"{metric}: Not found in history.")
        print(f"Final activation function: {final_activation}")
        
        # Example variables
        feature_csv = modelOuptuts + f"{Domain}_{basin_name}_model_featureList_summary.csv" 
        column_name = timestamp  
        feature_list = featureNames    
        new_column_df = pd.DataFrame({column_name: feature_list})
        
        # If the file already exists, load it and append the new column
        if os.path.exists(feature_csv):
            existing_df = pd.read_csv(feature_csv)
            # Reindex the existing dataframe 
            max_len = max(len(existing_df), len(new_column_df))
            existing_df = existing_df.reindex(range(max_len))
            new_column_df = new_column_df.reindex(range(max_len))
            # Combine horizontally
            combined_df = pd.concat([existing_df, new_column_df], axis=1)
        else:
            combined_df = new_column_df
        
        # Save back to CSV
        combined_df.to_csv(feature_csv, index=False)
        
        # add metrics to csv
        modelStatsCSV = modelOuptuts + f"{Domain}_{basin_name}_modelSummary_stats.csv"
        
        error_stats = {
            'ModelRun':[timestamp],
            'FeatureNum': [featNo],
            'Architecture': [architecture],
            'FinalActivation': [final_activation],
            'X_TrainShape': [x_trainShape[0]],
            'X_ValidShape': [x_validShape[0]],
            'RMSE': [best_metrics['val_masked_rmse']],
            'MSE': [best_metrics['val_masked_mse']], 
            'MAE': [best_metrics['val_masked_mae']]
            
        }
        
        df = pd.DataFrame(error_stats)
        
        # Append or write new file
        if os.path.exists(modelStatsCSV):
            df.to_csv(modelStatsCSV, mode='a', header=False, index=False)
        else:
            df.to_csv(modelStatsCSV, index=False)
        
        del X_train, X_valid, y_train, y_valid, X, y
        K.clear_session()
        gc.collect()
    
        del history 
        clear_memory()
    

C:\Users\etyrr\anaconda3\envs\basins-gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


modules imported
modules established
Baseline
MODEL OUTPUTS TO BE PRINTED TO THIS DOC
Processing year 2022
Processing year 2023
Processing year 2024

Shape of input data
feature shape: (66, 256, 256, 22)
target shape: (66, 65536)
22
[]
***
________________________________ Training and Validation Data Shapes ________________________________
Training data shape: (56, 256, 256, 22) (56, 65536)
Validation data shape: (10, 256, 256, 22) (10, 65536)
***
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 64)      12736     
                                                                 
 batch_normalization (BatchN  (None, 254, 254, 64)     256       
 ormalization)                                                   
                                                                 
 average_pooling2d (AverageP  (None, 127, 127, 64)     0         


ResourceExhaustedError: Graph execution error:

Detected at node 'gradient_tape/sequential/batch_normalization/FusedBatchNormGradV3' defined at (most recent call last):
    File "C:\Users\etyrr\anaconda3\envs\basins-gpu\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\etyrr\anaconda3\envs\basins-gpu\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\etyrr\anaconda3\envs\basins-gpu\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "C:\Users\etyrr\anaconda3\envs\basins-gpu\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "C:\Users\etyrr\anaconda3\envs\basins-gpu\lib\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "C:\Users\etyrr\anaconda3\envs\basins-gpu\lib\site-packages\tornado\platform\asyncio.py", line 195, in start
      self